In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model

img_a_inp = Input((28,28), name="img_a_inp" )
img_b_inp = Input((28,28), name="img_b_inp" )

In [ ]:
def get_cnn_block(depth):
  return Sequential([
      Conv2D(depth, 3,1),
      BatchNormalization(),
      ReLU()
  ])

In [ ]:
DEPTH = 64
cnn = Sequential([Reshape((28,28,1)),
                  get_cnn_block(DEPTH),
                  get_cnn_block(DEPTH*2),
                  get_cnn_block(DEPTH*4),
                  get_cnn_block(DEPTH*8),
                  GlobalAveragePooling2D(),
                  Dense(64, activation='relu')])

feature_a = cnn(img_a_inp)
feature_b = cnn(img_b_inp)

In [ ]:
concat = Concatenate()([feature_a, feature_b])

dense = Dense(64, activation='relu')(concat)

output = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[img_a_inp, img_b_inp], outputs=output)

model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

# 2. Save the model summary
with open('model_summary.txt', 'w') as f:
    # Redirect the print output to the file
    model.summary(print_fn=lambda x: f.write(x + '\n'))

print("Model architecture and summary saved.")

In [ ]:
import numpy as np
import random

random_indices = np.random.choice(x_train.shape[0], 300, replace=False)

x_train_sample, y_train_sample = x_train[random_indices], y_train[random_indices]

x_train_sample.shape, y_train_sample.shape

In [ ]:
import itertools

def make_paired_dataset(x,y):
  x_pairs, y_pairs = [], []
  tuples = [(x1, y1) for x1, y1 in zip(x,y)]

  for t in itertools.product(tuples, tuples):
    pair_A, pair_B = t
    img_a, label_a = t[0]
    img_b, label_b = t[1]

    new_label = int(label_a == label_b)

    x_pairs.append([img_a, img_b])
    y_pairs.append(new_label)

  x_pairs = np.array(x_pairs)
  y_pairs = np.array(y_pairs)

  return x_pairs, y_pairs

In [ ]:
x_train_pairs, y_train_pairs = make_paired_dataset(x_train_sample, y_train_sample)
x_train_pairs.shape, y_train_pairs.shape

In [ ]:
random_indices = np.random.choice(x_test.shape[0], 150, replace=False)

x_test_sample, y_test_sample = x_test[random_indices], y_test[random_indices]

x_test_sample.shape, y_test_sample.shape

In [ ]:
x_test_pairs, y_test_pairs = make_paired_dataset(x_test_sample, y_test_sample)
x_test_pairs.shape, y_test_pairs.shape

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=[x_train_pairs[:, 0, :, :],
             x_train_pairs[:, 1, :, :]],
          y=y_train_pairs,
          validation_data=([x_test_pairs[:, 0, :, :],
                           x_test_pairs[:, 1, :, :]], y_test_pairs),
          epochs=10,
          batch_size=32,
          callbacks=[es]
          )

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'history' is the output of model.fit()
plt.figure(figsize=(12, 5))

# Plotting loss
plt.subplot(1, 2, 1)
plt.plot(model.history.history['loss'], label='Training Loss')
plt.plot(model.history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting accuracy
plt.subplot(1, 2, 2)
plt.plot(model.history.history['accuracy'], label='Training Accuracy')
plt.plot(model.history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.savefig('training_history.png')
plt.show()

In [ ]:
model.save('Siamese_network-gpu.h5')

In [ ]:
img_a, img_b = x_test[0], x_test[17]
label_a , label_b = y_test[0], y_test[17]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(dpi=28)
plt.imshow(img_a)

In [ ]:
plt.figure(dpi=28)
plt.imshow(img_b)

In [ ]:
model.predict([img_a.reshape((1,28,28)), img_b.reshape((1,28,28))]).flatten()[0]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

img_a1, img_b1 = x_test[0], x_test[16]
label_a1 , label_b1 = y_test[0], y_test[16]
img_a2, img_b2 = x_test[31], x_test[17]
label_a2 , label_b2 = y_test[31], y_test[17]
img_a3, img_b3 = x_test[43], x_test[27]
label_a3 , label_b3 = y_test[43], y_test[27]
img_a4, img_b4 = x_test[0], x_test[17]
label_a4 , label_b4 = y_test[0], y_test[17]

# Sample input (use your actual image pairs)
img_pairs = [(img_a1, img_b1), (img_a2, img_b2), (img_a3, img_b3), (img_a4, img_b4)]  # Replace with your pairs

# Prepare the data for prediction
predictions = []
for img_a, img_b in img_pairs:
    score = model.predict([img_a.reshape((1, 28, 28)), img_b.reshape((1, 28, 28))]).flatten()[0]
    predictions.append(score)

# Create a plot of similarity scores
plt.figure(figsize=(10, 6))
plt.plot(range(len(predictions)), predictions, marker='o', linestyle='-')
plt.title('Similarity Scores for Image Pairs')
plt.xlabel('Image Pair Index')
plt.ylabel('Similarity Score')
plt.xticks(range(len(predictions)), [f'Pair {i+1}' for i in range(len(predictions))])  # Label pairs
plt.grid()
plt.savefig('similarity_scores_plot.png')  # Save the plot as an image file
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example pairs of images
image_pairs = [(img_a1, img_b1), (img_a2, img_b2), (img_a3, img_b3)]  # Add your image pairs here
similarity_scores = []

# Calculate similarity scores for each pair
for img_a, img_b in image_pairs:
    score = model.predict([img_a.reshape((1, 28, 28)), img_b.reshape((1, 28, 28))]).flatten()[0]
    similarity_scores.append(score)

# Create a bar plot for the similarity scores
plt.figure(figsize=(10, 5))
plt.bar(range(len(similarity_scores)), similarity_scores, color='blue')
plt.xticks(range(len(similarity_scores)), [f'Pair {i+1}' for i in range(len(similarity_scores))])
plt.title('Similarity Scores for Image Pairs')
plt.xlabel('Image Pairs')
plt.ylabel('Similarity Score')
plt.ylim(0, 1)  # Assuming scores are between 0 and 1
plt.grid(axis='y')
plt.savefig('similarity_scores.png')  # Save the plot as an image
plt.show()

In [ ]:
# Histogram of similarity scores
plt.figure(figsize=(10, 6))
plt.hist(predictions, bins=20, color='skyblue', edgecolor='black')
plt.title('Histogram of Similarity Scores')
plt.xlabel('Similarity Score')
plt.ylabel('Frequency')
plt.grid()
plt.savefig('similarity_scores_histogram.png')  # Save the plot as an image file
plt.show()

In [ ]:
# Box plot of similarity scores
plt.figure(figsize=(10, 6))
plt.boxplot(predictions, vert=False, patch_artist=True, boxprops=dict(facecolor='lightblue'))
plt.title('Box Plot of Similarity Scores')
plt.xlabel('Similarity Score')
plt.grid()
plt.savefig('similarity_scores_boxplot.png')  # Save the plot as an image file
plt.show()

In [ ]:
(label_a1, label_b1), (label_a2, label_b2), (label_a3, label_b3), (label_a4, label_b4)

In [ ]:
predictions

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Given true labels as tuples
true_labels = [(7, 9), (1, 7), (2, 4), (7, 7)]  # Example true labels

# Define the criteria for similarity
def label_similarity(label_pair):
    return 1 if label_pair[0] == label_pair[1] else 0

# Convert true labels to binary
true_labels_binary = [label_similarity(pair) for pair in true_labels]

# Example predicted probabilities from the model
predicted_probabilities = [0.000548531, 4.5291839e-07, 0.00013376739, 0.9968098]

# Convert probabilities to binary predictions
threshold = 0.5
predicted_labels = [1 if prob >= threshold else 0 for prob in predicted_probabilities]

# Compute confusion matrix
cm = confusion_matrix(true_labels_binary, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
             xticklabels=['Not Similar', 'Similar'], 
             yticklabels=['Not Similar', 'Similar'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_matrix.png')  # Save the plot as an image file
plt.show()